<a href="https://colab.research.google.com/github/ML-brooowss/ML/blob/main/Vx%20-%20everything%20till%20now" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Library
# !pip install sentence-transformers
# !pip install joblib
# !pip install collections
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid
from sklearn import datasets
from sklearn import svm
from collections import defaultdict
from joblib import Parallel, delayed
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split


# Recommender Systems
| Recommender Type     | Similarity Between | Based On           | Example Statement                                      |
|----------------------|--------------------|--------------------|--------------------------------------------------------|
| CF – Item-Item       | Items              | User behavior      | “You liked A, others who liked A also liked B”         |
| CF – User-User       | Users              | User behavior      | “People like you liked B, so you might too”            |
| Content-Based        | Items              | Item text/content  | “These books are similar in description/topic”         |
| Hybrid               | Items              | Content + Behavior | “You liked A; B is similar and liked by others too”    |


## Task 1: Exploring



#### Step 1:Get the data

In [2]:
# Load the datasets
# interactions = pd.read_csv('https://raw.githubusercontent.com/linneverh/MachineLearning/main/interactions_train.csv')
# items1 = pd.read_csv("https://raw.githubusercontent.com/linneverh/ML/refs/heads/main/embeddings_part1.csv")
# items2 = pd.read_csv("https://raw.githubusercontent.com/linneverh/ML/refs/heads/main/embeddings_part2.csv")
# items = pd.concat([items1, items2])

interactions = pd.read_csv('https://raw.githubusercontent.com/linneverh/MachineLearning/main/interactions_train.csv')
#FOR: Google enhanced & ISBN enhanced - author_date_title_subjects
items1 = pd.read_csv("https://media.githubusercontent.com/media/ML-brooowss/ML/refs/heads/main/final_items/author_date_title_subjects/embeddings_part1.csv")
items2 = pd.read_csv("https://media.githubusercontent.com/media/ML-brooowss/ML/refs/heads/main/final_items/author_date_title_subjects/embeddings_part2.csv")
items = pd.concat([items1, items2])

#rename columns
interactions = interactions.rename(columns={'u': 'user_id', 'i': 'book_id', 't': 'timestamp'})
items=items.rename(columns={'i':'book_id'})
# Display the first rows of the updated interactions DataFrame
display(interactions.head())
display(items.head())

# Display the first rows of each dataset
display(interactions.head())
display(items.head())

NameError: name 'pd' is not defined

In [4]:
# add embeddings to interactions table
interactions = pd.merge(interactions, items[['book_id', 'embedding']], on='book_id', how='left')

# Display the first rows of the updated interactions DataFrame to verify the merge
display(interactions.head())

,user_id,book_id,timestamp,embedding
0,4456,8581,1.687541e+09,"[0.01576336, -0.011471942, -0.069125324, -0.02..."
1,142,1964,1.679585e+09,"[0.04438236, -0.01765409, -0.040625393, -0.022..."
2,362,3705,1.706872e+09,"[0.050417468, -0.037109874, -0.025543252, -0.0..."
3,1809,11317,1.673533e+09,"[0.03253946, -0.03576064, -0.064230666, 0.0228..."
4,4384,1323,1.681402e+09,"[0.010108859, 0.00936277, -0.060633954, -0.036..."


In [5]:
def extract_column_titles(df):
    """
    Extracts column titles from a Pandas DataFrame and returns them as a list.
    """
    return list(df.columns)

# Extract and print column titles for each DataFrame
items_cols = extract_column_titles(items)

print("\nItems DataFrame Columns:")
items_cols


Items DataFrame Columns:


['Unnamed: 0',
 'CanonicalLink',
 'Description',
 'ISBN',
 'ImageLink',
 'Language',
 'PublishedDate',
 'Publisher',
 'Subjects',
 'Title',
 'author_clean',
 'google_api_title',
 'book_id',
 'title_clean',
 'title_description',
 'date_title_description',
 'author_title_description',
 'author_date_title_description',
 'author_date_title',
 'author_date_title_subjects',
 'author_title_subjects',
 'embedding']


#### Step 2: Check the Number of interactions, users and movies

In [6]:
n_users = interactions.user_id.nunique()
n_items = items.book_id.nunique()
print(f'Number of users = {n_users}, \n Number of books = {n_items} \n Number of interactions = {len(interactions)}')


Number of users = 7838, 
 Number of books = 15291 
 Number of interactions = 87047


In [7]:
n_items = items.book_id.nunique()
print(f'Number of books = {n_items}')

Number of books = 15291



#### Step 3: Split the Data into Training and Test Sets

In [8]:
# let's first sort the interactions by user and time stamp
interactions = interactions.sort_values(["user_id", "timestamp"])
interactions.head(100)

,user_id,book_id,timestamp,embedding
21035,0,0,1.680191e+09,"[-0.004826885, -0.0587869, -0.06438997, -0.007..."
28842,0,1,1.680783e+09,"[0.0041115503, -0.012976925, 0.0044452655, 0.0..."
3958,0,2,1.680801e+09,"[0.027354596, -0.025706276, -0.051459163, 0.00..."
29592,0,3,1.683715e+09,"[0.036929574, -0.0399203, -0.033997424, -0.006..."
6371,0,3,1.683715e+09,"[0.036929574, -0.0399203, -0.033997424, -0.006..."
...,...,...,...,...
20068,2,53,1.694861e+09,"[0.04563928, -0.053787332, -0.016430369, 0.006..."
12721,2,53,1.695226e+09,"[0.04563928, -0.053787332, -0.016430369, 0.006..."
86745,2,53,1.695226e+09,"[0.04563928, -0.053787332, -0.016430369, 0.006..."
19329,2,53,1.695226e+09,"[0.04563928, -0.053787332, -0.016430369, 0.006..."


In [9]:
interactions["pct_rank"] = interactions.groupby("user_id")["timestamp"].rank(pct=True, method='dense')
interactions.reset_index(inplace=True, drop=True)
interactions.head(10)

,user_id,book_id,timestamp,embedding,pct_rank
0,0,0,1.680191e+09,"[-0.004826885, -0.0587869, -0.06438997, -0.007...",0.04
1,0,1,1.680783e+09,"[0.0041115503, -0.012976925, 0.0044452655, 0.0...",0.08
2,0,2,1.680801e+09,"[0.027354596, -0.025706276, -0.051459163, 0.00...",0.12
3,0,3,1.683715e+09,"[0.036929574, -0.0399203, -0.033997424, -0.006...",0.16
4,0,3,1.683715e+09,"[0.036929574, -0.0399203, -0.033997424, -0.006...",0.20
5,0,4,1.686569e+09,"[0.05324783, -0.026807835, -0.009055429, 0.005...",0.24
6,0,5,1.687014e+09,"[0.0103662815, -0.05280713, -0.029626973, -0.0...",0.28
7,0,6,1.687014e+09,"[0.023781504, -0.054194607, -0.018097805, -0.0...",0.32
8,0,7,1.687014e+09,"[0.00092733064, -0.02754119, -0.0001447586, 0....",0.36
9,0,8,1.687260e+09,"[0.012236664, 0.005825913, -0.056410506, 0.015...",0.40


Now all remains to do is to pick the first 80% of the interactions of each user in the training set and the rest in the test set. We can do so using the `pct_rank` column.

In [10]:
train_data = interactions[interactions["pct_rank"] < 0.8]
test_data = interactions[interactions["pct_rank"] >= 0.8]

In [11]:
print("Training set size:", train_data.shape[0])
print("Testing set size:", test_data.shape[0])

Training set size: 65419
Testing set size: 21628


## Task 2: Creating User-Item Matrices for Implicit Feedback


In [12]:
print('number of users =', n_users, '| number of movies =', n_items)

number of users = 7838 | number of movies = 15291


#### Step 1: Define the Function to Create the Data Matrix


In [13]:
# Define a function to create the data matrix
def create_data_matrix(data, n_users, n_items):
    """
    This function returns a numpy matrix with shape (n_users, n_items).
    Each entry is a binary value indicating positive interaction.
    """
    data_matrix = np.zeros((n_users, n_items))
    data_matrix[data["user_id"].values, data["book_id"].values] = 1
    return data_matrix

#### Step 2: Create the Training and Testing Matrices

Now we can use the function to create matrices for both the training and testing data. Each cell in the matrix will show a 1 if there was a positive interaction in the training or testing data, and a 0 otherwise.

In [14]:
entire_data=create_data_matrix(interactions, n_users, n_items)

In [15]:
# Create the training and testing matrices
train_data_matrix = create_data_matrix(train_data, n_users, n_items)
test_data_matrix = create_data_matrix(test_data, n_users, n_items)

# Display the matrices to understand their structure
print('train_data_matrix')
print(train_data_matrix)
print("number of non-zero values: ", np.sum(train_data_matrix))
print('test_data_matrix')
print(test_data_matrix)
print("number of non-zero values: ", np.sum(test_data_matrix))


train_data_matrix
[[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
number of non-zero values:  49689.0
test_data_matrix
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
number of non-zero values:  19409.0


In [16]:
#give the dimensions of matrices
print("Train data matrix dimensions:", train_data_matrix.shape)
print("Test data matrix dimensions:", test_data_matrix.shape)

Train data matrix dimensions: (7838, 15291)
Test data matrix dimensions: (7838, 15291)


#### Basic Definitions

In [17]:
# Recommendation frame generation
def create_recommendation_table(user_predictions, top_n=10, separator=" "):
    """
    Creates a table of top-N recommendations for each user.

    Args:
        user_predictions (numpy.ndarray): Rows = users, columns = items. Predicted scores.
        top_n (int): Number of top recommendations per user.
        separator (str): Delimiter to join recommended book IDs.

    Returns:
        pandas.DataFrame: Columns = ['user_id', 'recommendation'].
    """
    recommendations = []
    num_users = user_predictions.shape[0]

    for user_id in range(num_users):
        top_items = np.argsort(user_predictions[user_id, :])[-top_n:][::-1]
        recommendations.append({
            'user_id': user_id,
            'recommendation': separator.join(map(str, top_items))
        })

    return pd.DataFrame(recommendations)

In [18]:
# Def for the precision_recall_at_k function
def precision_recall_at_k(prediction, ground_truth, k=10):
    """
    Calculates Precision@K and Recall@K for top-K recommendations.
    Parameters:
        prediction (numpy array): The predicted interaction matrix with scores.
        ground_truth (numpy array): The ground truth interaction matrix (binary).
        k (int): Number of top recommendations to consider.
    Returns:
        precision_at_k (float): The average precision@K over all users.
        recall_at_k (float): The average recall@K over all users.
    """
    num_users = prediction.shape[0]
    precision_at_k, recall_at_k = 0, 0

    for user in range(num_users):
        # TODO: Get the indices of the top-K items for the user based on predicted scores
        top_k_items = np.argsort(prediction[user, :])[-k:]

        # TODO: Calculate the number of relevant items in the top-K items for the user
        relevant_items_in_top_k = np.isin(top_k_items, np.where(ground_truth[user, :] == 1)[0]).sum()

        # TODO: Calculate the total number of relevant items for the user
        total_relevant_items = ground_truth[user, :].sum()

        # Precision@K and Recall@K for this user
        precision_at_k += relevant_items_in_top_k / k
        recall_at_k += relevant_items_in_top_k / total_relevant_items if total_relevant_items > 0 else 0

    # Average Precision@K and Recall@K over all users
    precision_at_k /= num_users
    recall_at_k /= num_users

    return precision_at_k, recall_at_k

In [19]:
# Create random splits def.
def random_split_per_user(interactions_df, test_size=0.2):
    train_list = []
    test_list = []
    for user_id, user_df in interactions_df.groupby('user_id'):
        train_df, test_df = train_test_split(user_df, test_size=test_size)
        train_list.append(train_df)
        test_list.append(test_df)
    return pd.concat(train_list), pd.concat(test_list)

In [20]:
# Define the function to predict interactions based on item similarity
def item_based_predict(interactions, similarity, epsilon=1e-9):
    """
    Predicts user-item interactions based on item-item similarity.
    Parameters:
        interactions (numpy array): The user-item interaction matrix.
        similarity (numpy array): The item-item similarity matrix.
        epsilon (float): Small constant added to the denominator to avoid division by zero.
    Returns:
        numpy array: The predicted interaction scores for each user-item pair.
    """
    # np.dot does the matrix multiplication. Here we are calculating the
    # weighted sum of interactions based on item similarity
    pred = similarity.dot(interactions.T) / (similarity.sum(axis=1)[:, np.newaxis] + epsilon)
    return pred.T  # Transpose to get users as rows and items as columns

#### TF-IDF
w. ['Publisher', 'Subjects', 'google_api_title', 'author_clean', 'ISBN']<br>
Mean Precision@10 = 0.0149 <br>
Mean Recall@10    = 0.091

w.

In [21]:
#TF-IDF

# STEP 1: Build and clean the combined text feature
text_fields = ["author_date_title_subjects"]
items['combined_text'] = items[text_fields].fillna('').agg(' '.join, axis=1)

# # STEP 2: Align items with those used in the train_data_matrix (e.g., by book_id)
# # to ensure the order of books in the TF-IDF matrix exactly matches the item columns in the collaborative filtering matrix, so similarity scores align correctly with item IDs.
# items_ordered = items.set_index('book_id').loc[range(train_data_matrix.shape[1])]

# # STEP 3: Compute TF-IDF matrix and cosine similarity
tfidf = TfidfVectorizer(stop_words='english', max_features=1000)
tfidf_matrix = tfidf.fit_transform(items['combined_text'])

# # Cosine similarity between item vectors
tfidf_sim = cosine_similarity(tfidf_matrix)


In [46]:
# Calculate the item-based predictions for positive interactions
item_tfidf_prediction = item_based_predict(entire_data, tfidf_sim)
print("Predicted Interaction Matrix:")
print(item_tfidf_prediction)
print(item_tfidf_prediction.shape)

Predicted Interaction Matrix:
[[1.93648001e-02 6.02324571e-03 3.70896020e-03 ... 0.00000000e+00
  1.24646513e-03 7.66639840e-04]
 [0.00000000e+00 3.78506739e-04 1.13538431e-03 ... 0.00000000e+00
  3.41357374e-03 2.97465019e-04]
 [4.83806086e-03 1.06604585e-03 3.20573129e-03 ... 4.21879565e-02
  3.94020317e-02 1.63662006e-03]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 3.06001250e-03
  1.66798383e-03 0.00000000e+00]
 [0.00000000e+00 4.18697117e-05 5.29821428e-05 ... 4.36731500e-03
  1.90399740e-03 5.27270444e-05]
 [0.00000000e+00 1.55179166e-04 3.30527775e-04 ... 0.00000000e+00
  7.63127322e-04 1.61388388e-04]]
(7838, 15291)


In [49]:
# Create df
item_tfidf_recommendations_df = create_recommendation_table(item_tfidf_prediction, top_n=10, separator=" ")

# Save and display
item_tfidf_recommendations_df.to_csv('item_tfidf_recommendations.csv', index=False)

print("\nItem-based Recommendations:")
display(item_tfidf_recommendations_df)


Item-based Recommendations:


,user_id,recommendation
0,0,12682 13261 16 2395 0 1583 4512 18 19 14725
1,1,14409 1178 13472 14405 7846 10787 767 5694 132...
2,2,13134 14824 15184 91 14898 14612 14605 14866 1...
3,3,7849 9526 351 2768 10707 5220 6128 4471 9388 8523
4,4,7603 5436 9779 197 11213 203 11063 15039 15285...
...,...,...
7833,7833,12105 14005 3224 8882 11860 3330 8085 14198 97...
7834,7834,7311 14555 14547 14559 14550 14551 14558 15081...
7835,7835,11087 8188 5037 8618 450 8128 4820 11482 3063 ...
7836,7836,14574 13134 14868 14824 91 14605 14558 14578 1...


In [50]:
precision_item_k, recall_item_k = precision_recall_at_k(item_tfidf_prediction, test_data_matrix, k=10)
print('Item-based EMBED Precision@K:', precision_item_k)
print('Item-based EMBED Recall@K:', recall_item_k)

Item-based EMBED Precision@K: 0.0390022965042103
Item-based EMBED Recall@K: 0.24758395243205233


In [ ]:
# Cross-validation setup

# Evaluate precision and recall for one run
def evaluate_one():
    train_df, test_df = random_split_per_user(interactions)
    train_matrix = create_data_matrix(train_df, n_users, n_items)
    prediction_matrix = item_based_predict(train_matrix, tfidf_sim)
    test_matrix = create_data_matrix(test_df, n_users, n_items)
    p_at_k, r_at_k = precision_recall_at_k(prediction_matrix, test_matrix, k=10)
    return p_at_k, r_at_k

# Run evaluations in parallel
results = Parallel(n_jobs=-1)(
    delayed(evaluate_one)()
)

# Extract and average
precisions, recalls = zip(*results)
mean_precision = np.mean(precisions)
mean_recall = np.mean(recalls)

# Print results
print(f"Mean Precision@10 = {mean_precision:.4f}")
print(f"Mean Recall@10    = {mean_recall:.4f}")


#### Google API similarity <BR>
Item-based EMBED Precision@K: 0.04866037254401807 <BR>
Item-based EMBED Recall@K: 0.2707247031495884

In [23]:
# # # Select only the item IDs in the training data matrix
# # train_item_ids = range(train_data_matrix.shape[1])

# # # Ensure correct item order by aligning to the item indices used in the train matrix
# # items_ordered = items.set_index('book_id').loc[train_item_ids]

# # Parse the embedding strings into numpy arrays
# items_ordered['embedding'] = items['embedding'].apply(lambda x: np.fromstring(x.strip('[]'), sep=','))

# # Drop rows with missing or malformed embeddings (if any)
# valid_items = items_ordered[items_ordered['embedding'].notna()].reset_index(drop=True)

# # Stack embeddings into a matrix
# embedding_matrix = np.vstack(valid_items['embedding'].values)

# # Compute cosine similarity
# embedding_sim = cosine_similarity(embedding_matrix)

# # Optional: Normalize similarity matrix
# embedding_sim = normalize(embedding_sim)
# Step 1: Convert embedding strings into NumPy arrays
items['embedding_vector'] = items['embedding'].apply(lambda x: np.fromstring(x.strip('[]'), sep=',') if isinstance(x, str) else x)

# Step 2: Drop rows where conversion failed (NaN or malformed vectors)
valid_items = items[items['embedding_vector'].apply(lambda x: isinstance(x, np.ndarray) and x.size > 0)].reset_index(drop=True)

# Step 3: Stack all vectors into a matrix
embedding_matrix = np.vstack(valid_items['embedding_vector'].values)

# Step 4: Normalize
embedding_matrix = normalize(embedding_matrix)

# Step 5: Compute cosine similarity
embedding_sim = cosine_similarity(embedding_matrix)

In [ ]:
# Calculate the item-based predictions for positive interactions
item_EMBED_prediction = item_based_predict(entire_data, embedding_sim)
print("Predicted Interaction Matrix:")
print(item_EMBED_prediction)
print(item_EMBED_prediction.shape)

In [ ]:
# CHECK PRECISION & RECALL NOT YET WITH CROSS-VALIDATION [OVERFITTING PROBLEM THOUGH]
precision_item_k, recall_item_k = precision_recall_at_k(item_EMBED_prediction, test_data_matrix, k=10)
print('Item-based EMBED Precision@K:', precision_item_k)
print('Item-based EMBED Recall@K:', recall_item_k)

In [ ]:
#Cross Validation
def evaluate_one(seed):
    train_df, test_df = random_split_per_user(interactions, seed=seed)
    train_matrix = create_data_matrix(train_df, n_users, n_items)

    # Compute similarity from current train split
    item_sim = cosine_similarity(train_matrix.T)
    prediction_matrix = item_based_predict(train_matrix, item_sim)

    # Evaluate on corresponding test set
    test_matrix = create_data_matrix(test_df, n_users, n_items)
    p_at_k, r_at_k = precision_recall_at_k(prediction_matrix, test_matrix, k=10)

    return p_at_k, r_at_k

# Run cross-validation
seeds = list(range(5))
results = Parallel(n_jobs=-1)(
    delayed(evaluate_one)(seed) for seed in seeds
)

# Unpack and average
precisions, recalls = zip(*results)
mean_precision = np.mean(precisions)
mean_recall = np.mean(recalls)

# Print results
print(f"Mean Precision@10 = {mean_precision:.4f}")
print(f"Mean Recall@10    = {mean_recall:.4f}")

#### BERT Similarity
Mean Precision@10 = 0.0272 <br>
Mean Recall@10    = 0.1760

In [86]:
# STEP 1: Combine text features
text_fields = ['author_date_title_subjects' ]
items['combined_text'] = items[text_fields].fillna('').agg(' '.join, axis=1)

# STEP 2: Align with train_data_matrix
items_ordered = items.set_index('book_id').loc[range(train_data_matrix.shape[1])]

# STEP 3: Load BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# STEP 4: Encode book texts into embeddings
bert_embeddings = model.encode(items_ordered['combined_text'].tolist(), show_progress_bar=True)

# STEP 5: Compute cosine similarity
bert_sim = cosine_similarity(bert_embeddings)

# STEP 6: Normalize (optional)
bert_sim = normalize(bert_sim)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/478 [00:00<?, ?it/s]

In [87]:
# Calculate the item-based predictions for positive interactions
item_bert_prediction = item_based_predict(train_data_matrix, bert_sim)
print("Predicted Interaction Matrix:")
print(item_bert_prediction)
print(item_bert_prediction.shape)

Predicted Interaction Matrix:
[[1.65212991e-03 1.79987189e-03 1.25307173e-03 ... 1.31800141e-03
  9.50959071e-04 1.45892263e-03]
 [6.84591608e-04 5.08573747e-04 5.55933745e-04 ... 5.51415288e-04
  7.12243857e-04 6.16411838e-04]
 [3.78045456e-03 2.35336103e-03 2.08550777e-03 ... 4.96338645e-03
  5.88617099e-03 2.34867493e-03]
 ...
 [1.39193990e-04 6.71983794e-05 7.94575106e-05 ... 3.33527517e-04
  2.85936802e-04 8.99615898e-05]
 [1.30111552e-04 1.07158576e-04 7.19629533e-05 ... 2.67730421e-04
  3.59789834e-04 6.73446640e-05]
 [1.24240592e-04 1.41484369e-04 1.23235837e-04 ... 1.72386522e-04
  2.75241823e-04 1.68135557e-04]]
(7838, 15291)


In [89]:
# Create recommendation
item_bert_recommendations_df = create_recommendation_table(item_bert_prediction, top_n=10, separator=" ")

# Save and display
item_bert_recommendations_df.to_csv('item_bert_recommendations.csv', index=False)

print("\nItem-based Recommendations:")
display(item_bert_recommendations_df)


Item-based Recommendations:


,user_id,recommendation
0,0,3928 14939 14359 8362 12906 14763 13353 9021 5...
1,1,725 14939 31 14476 3333 7154 30 61 11473 4853
2,2,725 15205 14605 14166 2142 13806 14559 14552 1...
3,3,725 13806 11379 14891 7154 14476 4853 3333 320...
4,4,3928 725 15205 14939 9297 12672 11418 5935 148...
...,...,...
7833,7833,14939 400 450 975 10997 7322 5999 14332 6661 6995
7834,7834,725 13806 15205 14605 14166 15081 2142 13952 1...
7835,7835,725 13806 14166 15081 14605 13952 4436 14559 1...
7836,7836,3928 14605 15205 14550 14559 14166 15081 15184...


In [90]:
p_at_k, r_at_k = precision_recall_at_k(item_bert_prediction, test_data_matrix, k=10)
print(f"Precision@10 = {p_at_k:.4f}")
print(f"Recall@10 = {r_at_k:.4f}")

Precision@10 = 0.0177
Recall@10 = 0.1151


In [ ]:
# Cross-validation setup
seeds = list(range(5))  # 5 random seeds for 5 train-test splits

# Evaluate precision and recall for one run
def evaluate_one(seed):
    train_df, test_df = random_split_per_user(interactions, seed=seed)
    train_matrix = create_data_matrix(train_df, n_users, n_items)
    prediction_matrix = item_based_predict(train_matrix, bert_sim)
    test_matrix = create_data_matrix(test_df, n_users, n_items)
    p_at_k, r_at_k = precision_recall_at_k(prediction_matrix, test_matrix, k=10)
    return p_at_k, r_at_k

# Run evaluations in parallel
results = Parallel(n_jobs=-1)(
    delayed(evaluate_one)(seed) for seed in seeds
)

# Extract and average
precisions, recalls = zip(*results)
mean_precision = np.mean(precisions)
mean_recall = np.mean(recalls)

# Print results
print(f"Mean Precision@10 = {mean_precision:.4f}")
print(f"Mean Recall@10    = {mean_recall:.4f}")

Mean Precision@10 = 0.0272
Mean Recall@10    = 0.1760


#### CF Item-based
Mean Precision@10 = 0.0585 <br>
Mean Recall@10    = 0.2823

In [24]:
# Compute the item-item similarity matrix
item_similarity = cosine_similarity(entire_data.T)
print("Item-Item Similarity Matrix:")
print(item_similarity)
print(item_similarity.shape)

Item-Item Similarity Matrix:
[[1.         0.40824829 0.33333333 ... 0.         0.         0.        ]
 [0.40824829 1.         0.40824829 ... 0.         0.         0.        ]
 [0.33333333 0.40824829 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
(15291, 15291)


In [ ]:
# Calculate the item-based predictions for positive interactions
item_prediction = item_based_predict(entire_data, item_similarity)
print("Predicted Interaction Matrix:")
print(item_prediction)
print(item_prediction.shape)

In [60]:
# Create recommendation
item_CF_recommendations_df = create_recommendation_table(item_prediction, top_n=10, separator=" ")

# Save and display
item_CF_recommendations_df.to_csv('item_CF_recommendations.csv', index=False)

print("\nItem-based Recommendations:")
display(item_CF_recommendations_df)


Item-based Recommendations:


,user_id,recommendation
0,0,2 3 1 0
1,1,2 3 1 0
2,2,2 3 1 0
3,3,2 3 1 0
4,4,2 3 1 0
...,...,...
87042,87042,2 3 1 0
87043,87043,2 3 1 0
87044,87044,2 3 1 0
87045,87045,2 3 1 0


In [27]:
p_at_k, r_at_k = precision_recall_at_k(item_prediction, test_data_matrix, k=10)
print(f"Precision@10 = {p_at_k:.4f}")
print(f"Recall@10 = {r_at_k:.4f}")

Precision@10 = 0.0557
Recall@10 = 0.2640


In [ ]:
#Cross Validation
def evaluate_one(seed):
    train_df, test_df = random_split_per_user(interactions, seed=seed)
    train_matrix = create_data_matrix(train_df, n_users, n_items)

    # Compute similarity from current train split
    item_sim = cosine_similarity(train_matrix.T)
    prediction_matrix = item_based_predict(train_matrix, item_sim)

    # Evaluate on corresponding test set
    test_matrix = create_data_matrix(test_df, n_users, n_items)
    p_at_k, r_at_k = precision_recall_at_k(prediction_matrix, test_matrix, k=10)

    return p_at_k, r_at_k

# Run cross-validation
seeds = list(range(5))
results = Parallel(n_jobs=-1)(
    delayed(evaluate_one)(seed) for seed in seeds
)

# Unpack and average
precisions, recalls = zip(*results)
mean_precision = np.mean(precisions)
mean_recall = np.mean(recalls)

# Print results
print(f"Mean Precision@10 = {mean_precision:.4f}")
print(f"Mean Recall@10    = {mean_recall:.4f}")

Mean Precision@10 = 0.0585
Mean Recall@10    = 0.2823


#### Hybrid Model testing
between:
1.   item_similarity
2.   bert_sim
3.   tfidf_sim
4.   embedding_sim
5.  user_sim




Alpha tuning + cross validaing w. random train-test splits

In [28]:
#running in parallell
alpha_values = np.linspace(0, 1, 6)
seeds = list(range(5))

# One evaluation run for a single (alpha, seed) pair
def evaluate_one(alpha, seed):
    train_df, test_df = random_split_per_user(interactions, seed=seed)
    train_matrix = create_data_matrix(train_df, n_users, n_items)
    hybrid_sim = alpha * item_similarity + (1 - alpha) * embedding_sim #here change sim to sim we want!
    prediction_matrix = item_based_predict(train_matrix, hybrid_sim)
    test_matrix = create_data_matrix(test_df, n_users, n_items)
    p_at_k = precision_recall_at_k(prediction_matrix, test_matrix, k=10)
    return (alpha, p_at_k)

# Run all combinations in parallel
results = Parallel(n_jobs=-1)(
    delayed(evaluate_one)(alpha, seed)
    for alpha in alpha_values
    for seed in seeds
)

# Aggregate results by alpha
by_alpha = defaultdict(list)
for alpha, precision in results:
    by_alpha[alpha].append(precision)

# Report means
for alpha in sorted(by_alpha.keys()):
    mean_p = np.mean(by_alpha[alpha])
    print(f"alpha = {alpha:.1f} → mean Precision@10 = {mean_p:.4f}")


alpha = 0.0 → mean Precision@10 = 0.1760
alpha = 0.2 → mean Precision@10 = 0.3343
alpha = 0.4 → mean Precision@10 = 0.3352
alpha = 0.6 → mean Precision@10 = 0.3365
alpha = 0.8 → mean Precision@10 = 0.3363
alpha = 1.0 → mean Precision@10 = 0.2966


In [50]:
train_kaggle_matrix = create_data_matrix(interactions, n_users, n_items)

In [1]:
hybrid_CFItemxtfidfxembedding = 0.2 * tfidf_sim + 0.2 * embedding_sim + (1 - 0.4) * item_similarity

NameError: name 'tfidf_sim' is not defined

In [26]:
# Calculate the item-based predictions for positive interactions
item_prediction = item_based_predict(entire_data, hybrid_CFItemxtfidfxembedding)
print("Predicted Interaction Matrix:")
print(item_prediction)
print(item_prediction.shape)

Predicted Interaction Matrix:
[[0.00331134 0.00353645 0.0031293  ... 0.00160413 0.00158929 0.00164699]
 [0.00069335 0.00066823 0.000727   ... 0.00071527 0.00090591 0.00066635]
 [0.00331012 0.0030938  0.00320296 ... 0.00398805 0.00569799 0.00309461]
 ...
 [0.00017953 0.00016454 0.00016855 ... 0.00024338 0.00029803 0.00017694]
 [0.00012251 0.00011847 0.00011821 ... 0.00019012 0.00024773 0.00011586]
 [0.00018362 0.00018127 0.00018718 ... 0.00020422 0.00024271 0.00018653]]
(7838, 15291)


In [27]:
# Create recommendation
hybrid_CFItemXtfidfXembedding_df = create_recommendation_table(item_prediction, top_n=10, separator=" ")

# Save and display
hybrid_CFItemXtfidfXembedding_df.to_csv('hybrid_CFItemXtfidfXembedding.csv', index=False)

####CF User-based
Mean Precision@10 = 0.0612 <br>
Mean Recall@10    = 0.3167

In [81]:
# Compute the user-user similarity matrix
user_similarity = cosine_similarity(entire_data)
print("User-User Similarity Matrix:")
print(user_similarity)

# Check the shape as a sanity check
print("Shape of User Similarity Matrix:", user_similarity.shape)

User-User Similarity Matrix:
[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.08084521]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.08084521 ... 0.         0.         1.        ]]
Shape of User Similarity Matrix: (7838, 7838)


In [83]:
# Define the function to predict interactions based on user similarity
def user_based_predict(interactions, similarity, epsilon=1e-9):
    """
    Predicts user-item interactions based on user-user similarity.
    Parameters:
        interactions (numpy array): The user-item interaction matrix.
        similarity (numpy array): The user-user similarity matrix.
        epsilon (float): Small constant added to the denominator to avoid division by zero.
    Returns:
        numpy array: The predicted interaction scores for each user-item pair.
    """
    # Calculate the weighted sum of interactions based on user similarity
    pred = similarity.dot(interactions) / (np.abs(similarity).sum(axis=1)[:, np.newaxis] + epsilon)
    return pred

# Calculate the user-based predictions for positive interactions
user_prediction = user_based_predict(entire_data, user_similarity)
print("Predicted Interaction Matrix (User-Based):")
print(user_prediction)
print(user_prediction.shape)

Predicted Interaction Matrix (User-Based):
[[0.12083887 0.12253831 0.12798326 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.00421191 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
(7838, 15291)


In [85]:
# Create recommendation
user_CF_recommendations_df = create_recommendation_table(user_prediction, top_n=10, separator=" ")

# Save and display
user_CF_recommendations_df.to_csv('user_CF_recommendations.csv', index=False)

print("\nuser-based Recommendations:")
display(user_CF_recommendations_df)


user-based Recommendations:


,user_id,recommendation
0,0,13 4 12 23 15 14 11 8 5 9
1,1,38 39 30 31 34 36 37 32 33 29
2,2,46 58 49 56 53 91 64 87 45 71
3,3,149 169 163 167 128 133 143 40 139 165
4,4,203 198 207 205 195 202 193 191 199 201
...,...,...
7833,7833,7760 975 7322 7306 611 8086 1130 9610 5838 11291
7834,7834,1367 13891 7128 8999 15276 3055 101 2125 10651...
7835,7835,3055 6791 4820 11126 8369 8999 9719 53 1367 15062
7836,7836,3471 14550 14552 611 15065 3470 8999 618 14557...


In [39]:
p_at_k, r_at_k = precision_recall_at_k(user_prediction, test_data_matrix, k=10)
print(f"Precision@10 = {p_at_k:.4f}")
print(f"Recall@10 = {r_at_k:.4f}")

Precision@10 = 0.0565
Recall@10 = 0.2905


In [40]:
#Cross Validation
def evaluate_one(seed):
    train_df, test_df = random_split_per_user(interactions, seed=seed)
    train_matrix = create_data_matrix(train_df, n_users, n_items)

    # Compute similarity from current train split
    user_sim = cosine_similarity(train_matrix)
    prediction_matrix = user_based_predict(train_matrix, user_sim)

    # Evaluate on corresponding test set
    test_matrix = create_data_matrix(test_df, n_users, n_items)
    p_at_k, r_at_k = precision_recall_at_k(prediction_matrix, test_matrix, k=10)

    return p_at_k, r_at_k

# Run cross-validation
seeds = list(range(5))
results = Parallel(n_jobs=-1)(
    delayed(evaluate_one)(seed) for seed in seeds
)

# Unpack and average
precisions, recalls = zip(*results)
mean_precision = np.mean(precisions)
mean_recall = np.mean(recalls)

# Print results
print(f"Mean Precision@10 = {mean_precision:.4f}")
print(f"Mean Recall@10    = {mean_recall:.4f}")

KeyboardInterrupt: 

#### Hybrid Model testing
between:
1.   item_similarity
2.   bert_sim
3.   tfidf_sim
4.   embedding_sim
5.  user_sim

<table border="1" style="border-collapse: collapse; text-align: left;">
  <caption style="font-size: 18px; font-weight: bold; padding: 10px;"> </caption>
  <thead>
    <tr>
      <th>Blending Stage</th>
      <th>Use When...</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Prediction-level</td>
      <td>Models use different mechanics (CF, BERT, popularity)</td>
    </tr>
    <tr>
      <td>Similarity-level</td>
      <td>All models are similarity-based and aligned (e.g., TF-IDF + BERT)</td>
    </tr>
  </tbody>
</table>





Alpha tuning + cross validaing w. random train-test splits --> only for item-item bc. need same matrix shape

In [42]:
#running in parallell
alpha_values = np.linspace(0, 1, 6)
seeds = list(range(5))

# One evaluation run for a single (alpha, seed) pair
def evaluate_one(alpha, seed):
    train_df, test_df = random_split_per_user(interactions, seed=seed)
    train_matrix = create_data_matrix(train_df, n_users, n_items)
    hybrid_sim = alpha * user_similarity + (1 - alpha) * item_similarity #here change sim to sim we want!
    prediction_matrix = item_based_predict(train_matrix, hybrid_sim)
    test_matrix = create_data_matrix(test_df, n_users, n_items)
    p_at_k = precision_recall_at_k(prediction_matrix, test_matrix, k=10)
    return (alpha, p_at_k)

# Run all combinations in parallel
results = Parallel(n_jobs=-1)(
    delayed(evaluate_one)(alpha, seed)
    for alpha in alpha_values
    for seed in seeds
)

# Aggregate results by alpha
by_alpha = defaultdict(list)
for alpha, precision in results:
    by_alpha[alpha].append(precision)

# Report means
for alpha in sorted(by_alpha.keys()):
    mean_p = np.mean(by_alpha[alpha])
    print(f"alpha = {alpha:.1f} → mean Precision@10 = {mean_p:.4f}")


ValueError: operands could not be broadcast together with shapes (7838,7838) (15291,15291) 

In [ ]:
train_kaggle_matrix = create_data_matrix(interactions, n_users, n_items)

In [ ]:
hybrid_sim_tf_item = 0.2 * tfidf_sim + (1 - 0.2) * item_similarity

In [ ]:
# Calculate the item-based predictions for positive interactions
item_prediction = item_based_predict(train_kaggle_matrix, hybrid_sim_tf_item)
print("Predicted Interaction Matrix:")
print(item_prediction)
print(item_prediction.shape)

Predicted Interaction Matrix:
[[2.65886616e-01 2.90080218e-01 2.65197196e-01 ... 0.00000000e+00
  8.79440059e-04 9.65572629e-04]
 [0.00000000e+00 2.06567322e-04 5.75873753e-04 ... 0.00000000e+00
  1.96614989e-03 3.32813416e-04]
 [1.42369587e-03 9.08401287e-04 1.61234917e-03 ... 3.19873920e-02
  3.34122545e-02 1.84868647e-03]
 ...
 [0.00000000e+00 3.15668202e-05 0.00000000e+00 ... 3.87327548e-03
  1.64930141e-03 0.00000000e+00]
 [0.00000000e+00 1.70350220e-05 4.29579013e-05 ... 3.05201971e-03
  1.58728431e-03 4.78381063e-05]
 [0.00000000e+00 1.36897791e-04 7.29899057e-05 ... 0.00000000e+00
  7.45820237e-04 2.19869433e-04]]
(7838, 15291)


In [ ]:
# Create recommendation
item_hybrid_tfidf_item_recommendations_df = create_recommendation_table(item_prediction, top_n=10, separator=" ")

# Save and display
item_hybrid_tfidf_item_recommendations_df.to_csv('item_hybrid1_recommendations.csv', index=False)

In [ ]:
### step 1.1: Combine CF + BERT
content_sim = 0.5 * tfidf_sim + 0.5 * embed_sim

In [ ]:
### step 1.2: Combine CF + TF-IDF
content_sim = 0.5 * tfidf_sim + 0.5 * embed_sim

In [ ]:
### step 1.3: Combine CF + Google API
content_sim = 0.5 * tfidf_sim + 0.5 * embed_sim

In [ ]:
### step 2.1: Combine TF-IDF + BERT
content_sim = 0.5 * tfidf_sim + 0.5 * embed_sim

In [ ]:
### step 2.2 : Combine TF-IDF + GOOGLE API
content_sim = 0.5 * tfidf_sim + 0.5 * embed_sim

In [ ]:
### step 2: Combine BERT + GOOGLE API
content_sim = 0.5 * tfidf_sim + 0.5 * embed_sim

#### Random tries - doesn't provide better result

In [44]:
# Trying the amazon model [content, item & popularity]
# 1. Item-based CF prediction
item_cf_pred = item_based_predict(train_data_matrix, item_similarity)

# 2. Content-based prediction (e.g., from BERT or TF-IDF)
content_pred = item_based_predict(train_data_matrix, embedding_sim)

# 3. Popularity score: frequency of item interactions
item_popularity = train_data_matrix.sum(axis=0)  # shape: (n_items,)
popularity_pred = np.tile(item_popularity, (train_data_matrix.shape[0], 1))

# 4. Normalize all predictions (important for blending)
from sklearn.preprocessing import MinMaxScaler

def normalize_rows(matrix):
    scaler = MinMaxScaler()
    return scaler.fit_transform(matrix)

item_cf_pred = normalize_rows(item_cf_pred)
content_pred = normalize_rows(content_pred)
popularity_pred = normalize_rows(popularity_pred)

# 5. Weighted blend (adjust alpha/beta/gamma)
alpha, beta, gamma = 0.4, 0.4, 0.2
hybrid_pred = alpha * item_cf_pred + beta * content_pred + gamma * popularity_pred

#tune
alpha, beta, gamma

(0.4, 0.4, 0.2)

In [45]:
import itertools

# Step 1: Define possible values for alpha, beta, gamma (sum to 1)
values = np.linspace(0, 1, 11)  # e.g., 0.0, 0.1, ..., 1.0
triplets = [
    (a, b, 1 - a - b)
    for a, b in itertools.product(values, values)
    if 0 <= 1 - a - b <= 1
]

# Step 3: Evaluate each combination
best_precision = 0
best_params = None

for alpha, beta, gamma in triplets:
    hybrid_pred = alpha * item_cf_pred + beta * content_pred + gamma * popularity_pred
    p_at_k, _ = precision_recall_at_k(hybrid_pred, test_data_matrix, k=10)

    if p_at_k > best_precision:
        best_precision = p_at_k
        best_params = (alpha, beta, gamma)

# Step 4: Print best combination
print(f"Best alpha, beta, gamma = {best_params}")
print(f"Best Precision@10 = {best_precision:.4f}")


Best alpha, beta, gamma = (np.float64(0.1), np.float64(0.9), np.float64(0.0))
Best Precision@10 = 0.0594


In [46]:
hybrid_pred = 0.1 * item_prediction + 0.9 * _pred + 0 * popularity_pred

In [48]:
# Create recommendation
user_CF_recommendations_df = create_recommendation_table(hybrid_pred, top_n=10, separator=" ")

# Save and display
user_CF_recommendations_df.to_csv('user_CF_recommendations.csv', index=False)

print("\nuser-based Recommendations:")
display(user_CF_recommendations_df)



user-based Recommendations:


,user_id,recommendation
0,0,1 7 17 18 2 0 8 5 6 16
1,1,31 33 32 29 36 35 2988 34 37 2442
2,2,44 76 45 78 75 59 82 79 54 53
3,3,132 155 151 144 139 154 131 116 140 156
4,4,192 203 191 202 197 204 195 193 198 200
...,...,...
7833,7833,975 7322 10460 8670 5116 3167 640 5119 3245 1588
7834,7834,15276 7128 13890 13891 7129 13414 114 13952 10...
7835,7835,3055 4820 15247 13016 6317 9957 11126 6223 148...
7836,7836,14550 3471 14557 15184 14560 14851 14618 11820...


In [77]:
hybrid_pred = 0.1 * item_prediction + 0.9 * item_EMBED_prediction + 0 * popularity_pred

In [78]:
# Create recommendation
item_CF_recommendations_df = create_recommendation_table(hybrid_pred, top_n=10, separator=" ")

# Save and display
item_CF_recommendations_df.to_csv('item_CF_recommendations.csv', index=False)

print("\nItem-based Recommendations:")
display(item_CF_recommendations_df)


Item-based Recommendations:


,user_id,recommendation
0,0,1 24 21 2 20 0 17 7 6 14
1,1,31 33 35 29 13434 32 36 2988 6003 12857
2,2,80 94 92 76 50 52 79 72 57 59
3,3,157 132 145 140 134 116 155 162 151 168
4,4,192 204 202 200 203 191 194 205 196 195
...,...,...
7833,7833,12575 7322 10460 5121 975 7760 5122 1978 13458...
7834,7834,15276 13890 7128 13891 15255 2154 15226 13808 ...
7835,7835,4820 6317 13626 9957 4061 14875 3055 14471 147...
7836,7836,15065 14550 14619 15184 14560 14092 14618 1182...
